In [10]:
# To get onnx file, run in terminal:
# python setup.py test --addopts "-k test_brevitas_quartznet" 
import numpy as np
from finn.util.visualization import showInNetron
import onnx
from finn.core.modelwrapper import ModelWrapper

file_name = '/tmp/quartznet.onnx'
showInNetron(file_name)



Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet.onnx' at http://0.0.0.0:8081


In [11]:
from finn.util.basic import *

model = ModelWrapper(file_name)
attributes=[]
for n in model.graph.node:
    if n.op_type=="Conv":
        for i in range(len(n.attribute)):
            attributes.append(n.attribute[i].name)
attributes = set(attributes)
print(attributes)

{'kernel_shape', 'dilations', 'strides', 'group', 'pads'}


# Extract the parameters of the conv node

In [12]:
model = ModelWrapper(file_name)
num_of_convs=0
param_dict={}
param_dict["dilations"]=[]
param_dict["kernel_shape"]=[]
param_dict["pads"]=[]
param_dict["strides"]=[]
param_dict["group"]=[]
attributes=[]
for n in model.graph.node:
    if n.op_type=="Conv":
        num_of_convs=num_of_convs+1
        #print(n.attribute)
        for i in range(len(n.attribute)):
            attributes.append(n.attribute[i].name)
        dilations = get_by_name(n.attribute, "dilations","name").ints[0]
        kernel_shape = get_by_name(n.attribute, "kernel_shape","name").ints[0]
        pads = [get_by_name(n.attribute, "pads", "name").ints[0], get_by_name(n.attribute, "pads", "name").ints[1]]
        strides = get_by_name(n.attribute, "strides", "name").ints[0]
        group = get_by_name(n.attribute, "group", "name").i
        param_dict["dilations"].append(dilations)
        param_dict["kernel_shape"].append(kernel_shape)
        param_dict["pads"].append(pads)
        param_dict["strides"].append(strides)
        param_dict["group"].append(group)
print(num_of_convs)

171


In [13]:
param_dict["dilations"]=set(param_dict["dilations"])
param_dict["kernel_shape"]=set(param_dict["kernel_shape"])
val, val_idx = np.unique(param_dict["pads"], return_index=True, axis=0)
param_dict["pads"]= [param_dict["pads"][el] for el in val_idx]
param_dict["strides"]=set(param_dict["strides"])
param_dict["group"]=set(param_dict["group"])

for key, value in param_dict.items():
    print("{} \t {}" .format(key,value))

dilations 	 {1, 2}
kernel_shape 	 {33, 1, 39, 75, 51, 87, 63}
pads 	 [[0, 0], [16, 16], [19, 19], [25, 25], [31, 31], [37, 37], [86, 86]]
strides 	 {1, 2}
group 	 {64, 1, 256, 512}


# Apply transformation
### Testing how to convert NCH -> NCH1

In [14]:
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.general import RemoveUnusedTensors
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.infer_datatypes import InferDataTypes
from finn.util.basic import *
from finn.custom_op import *
from onnx import helper
#import onnx

model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferShapes())
#model = model.transform(InferDataTypes())
model = model.transform(RemoveUnusedTensors())
model.save("/tmp/quartznet_modified.onnx")
showInNetron("/tmp/quartznet_modified.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet_modified.onnx' at http://0.0.0.0:8081


In [15]:
def find_invalid_nodes(model):
    """ Verifies whether the graph contains valid nodes """
    valid_nodes = ["Add", "Mul", "BatchNormalization", "MultiThreshold", "Conv", "Transpose", "LogSoftmax", "ArgMax"]
    invalid_nodes = []
    for n in model.graph.node:
        node_op_type = n.op_type
        if node_op_type in valid_nodes:
            continue
        else:
            invalid_nodes.append(node_op_type)
    if not invalid_nodes: # if there are no invalid nodes
        return True
    else:
        raise Exception("Nodes {} are not supported in this transformation".format(invalid_nodes))

def convert_3d_to_4d_tensors(model):
    """ Converts 3D tensors (input, value_info, output) to 4D tensors """
    tensor_names = {}
    ## Inputs
    for t in model.graph.input:
        tensor_name = t.name
        tensor_type = t.type.tensor_type.elem_type
        tensor_shape = model.get_tensor_shape(tensor_name)
        tensor_names[tensor_name] = [tensor_type]
        tensor_names[tensor_name].append(tensor_shape)

    ## Initializers
    initializer_names = []
    for i in model.graph.initializer:
        initializer_names.append(i.name)

    ## Value infos
    for t in model.graph.value_info:
        tensor_name = t.name
        if tensor_name in initializer_names:
            continue
        else:
            tensor_type = t.type.tensor_type.elem_type
            tensor_shape = model.get_tensor_shape(tensor_name)
            tensor_names[tensor_name] = [tensor_type]
            tensor_names[tensor_name].append(tensor_shape)

    ## Outputs
    for t in model.graph.output:
        tensor_name = t.name
        tensor_type = t.type.tensor_type.elem_type
        tensor_shape = model.get_tensor_shape(tensor_name)
        tensor_names[tensor_name] = [tensor_type]
        tensor_names[tensor_name].append(tensor_shape)

    ## Find tensors that are the output of nodes that reduce the dimension
    tensors_reduced_dimension = []
    for n in model.graph.node:
        if n.op_type=="ArgMax":
            keep_dims = get_by_name(n.attribute, "keepdims", "name").i
            if keep_dims == 0:
                node_out = n.output
                for n_o in node_out:
                    tensors_reduced_dimension.append(n_o)

    ## Change format of each value_info + input + output tensors
    for k, v in tensor_names.items():
        tensor_type = v[0]
        shape = v[1]
        ## Add extra dimension for tensors:
        # 1) Have 3 dimensions (NCH -> NCH1)
        # 2) Tensors following operations that reduce their dimension: {Argmax, ...}
        if len(shape)==3 or k in tensors_reduced_dimension:
            shape.append(1)
            model.set_tensor_shape(k, shape, tensor_type)
        else:
            continue

def make_node_4d_compatible(model):
    """ Each node from the list of supported nodes is made compatible with 4D tensors"""
    for n in model.graph.node:
        node_op_type = n.op_type
        if node_op_type == "Transpose":
            perm = get_by_name(n.attribute,"perm", "name").ints
            if len(perm) == 3: # Meaning transpose was on 3D tensor
                perm.append(3) # append 4th dimension
        elif node_op_type == "ArgMax" or node_op_type == "LogSoftMax":
            axis = get_by_name(n.attribute, "axis", "name")
            if axis.i == -1:
                axis.i = 2 # argmax is now on the second-to-last axis
        elif node_op_type == "Conv":
            dilations = get_by_name(n.attribute, "dilations", "name").ints
            kernel_shape = get_by_name(n.attribute, "kernel_shape", "name").ints
            pads = get_by_name(n.attribute, "pads", "name").ints
            strides = get_by_name(n.attribute, "strides", "name").ints
            if len(dilations) == 1: # we must add another dimension to it
                dilations.append(dilations[0]) # only equal dilation value along each spatial axis is supported
            if len(kernel_shape) == 1: # we must add another dimension to it
                kernel_shape.append(1)
            if len(pads) == 2: # pads = ([x1_begin, x1_end] -->) [x1_begin, x2_begin, x1_end, x2_end]
                pads.insert(1, 0)
                pads.append(0)
            if len(strides) == 1: # strides = [stride_H, stride_W]
                strides.append(1)

def create_4d_initializers_conv_mul_add_node(model):
    """ Conv, Mul and Add nodes are made compatible with 4D input tensors """
    initializers = {}
    for i in model.graph.initializer:
        init_name = i.name
        if "Conv" in init_name:
            init_dim = i.dims
            init_dtype = i.data_type
            initializers[init_name] = [init_dtype]
            initializers[init_name].append(init_dim)
        elif init_name[0:4] == "Mul_":
            init_dim = i.dims
            if len(i.dims) == 3:
                init_dtype = i.data_type
                initializers[init_name] = [init_dtype]
                initializers[init_name].append(init_dim)
        elif "Add" in init_name:
            init_dim = i.dims
            if len(i.dims) == 3:
                init_dtype = i.data_type
                initializers[init_name] = [init_dtype]
                initializers[init_name].append(init_dim)
        else:
            continue

    for k, v in initializers.items():
        init_dtype = v[0]
        init_shape = v[1]
        if len(init_shape) == 3:
            # Change shape NCH -> NCH1
            init_shape.append(1)
            shape_init = model.get_initializer(k).shape
            model.set_tensor_shape(k, init_shape, init_dtype)
            

model = ModelWrapper("/tmp/quartznet_modified.onnx")            
assert find_invalid_nodes(model)
convert_3d_to_4d_tensors(model)
make_node_4d_compatible(model)
create_4d_initializers_conv_mul_add_node(model)

model.save("/tmp/quartznet_modified_4d.onnx")    
showInNetron("/tmp/quartznet_modified_4d.onnx")  


Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet_modified_4d.onnx' at http://0.0.0.0:8081


In [16]:
model = ModelWrapper("/tmp/quartznet_modified_4d.onnx")
model = model.transform(InferDataTypes())
model = model.transform(InferShapes())
model.save("/tmp/quartznet_modified_4d_inferred_shapes.onnx")    
showInNetron("/tmp/quartznet_modified_4d_inferred_shapes.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet_modified_4d_inferred_shapes.onnx' at http://0.0.0.0:8081


In [3]:
##
## UNDER CONSTRUCTION
##

# repetitive_nodes = []
# rep = []
# i = 0
# for n in model.graph.node:
#     is_fork = model.is_fork_node(n)
#     if is_fork is True:
#         i = (i+1)%2
#     if i == 1:
#         rep.append(n)
#     if i == 0:
#         if rep: # rep is non-empty
#             repetitive_nodes.append(rep)
#             break
            
# print(repetitive_nodes)

# inp = []
# out = []
# value_info = []
# initializers = []
# for i, n in enumerate(repetitive_nodes[0]):
#     if i == 0: # input node
#         inp.append(n.input[0])
#         initializers.append(n.input[1])
#     elif i == len(repetitive_nodes[0])-1: # output node
#         value_info.append(n.input[0])
#         initializers.append(n.input[1])
#         out.append(n.output[0])
#     else: # intermediate node
#         value_info.append(n.input[0])
#         initializers.append(n.input[1])
    


In [37]:
## Basic transformations (leave commented)

#from finn.util.basic import *
#from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
#from finn.transformation.infer_shapes import InferShapes
#from finn.transformation.infer_datatypes import InferDataTypes
#from finn.transformation.fold_constants import FoldConstants

#file_name = "/tmp/quartznet.onnx"
#model = ModelWrapper(file_name)
#model = model.transform(InferShapes())
#model = model.transform(FoldConstants())
#model = model.transform(GiveUniqueNodeNames())
#model = model.transform(GiveReadableTensorNames())
#model = model.transform(InferDataTypes())
#model = model.transform(RemoveStaticGraphInputs())
#model.save("/tmp/quartznet_cleaned.onnx")
#file_name = "/tmp/quartznet_cleaned.onnx"

#showInNetron(file_name)

# Execute graph and compare results

In [17]:
import finn.core.onnx_exec as oxe

def make_unit_input(model):
    """ Creates numpy array that matches the input tensor shape """
    i_shape = []
    input_node = model.graph.input[0]
    input_node_shape = input_node.type.tensor_type.shape.dim
    input_node_name = input_node.name

    for d in input_node_shape:
        i_shape.append(d.dim_value)

    i_val = np.ones(i_shape).astype(np.float32)

    input_dict = {input_node_name: i_val}
    
    return input_dict

def execute_unit_input_test(model):
    """ Executes unit input test on the supplied model. Return value is the value of the output of the graph"""
    output_node_name = model.graph.output[0].name
    input_dict = make_unit_input(model)
    output_dict = oxe.execute_onnx(model, input_dict)
    expected = output_dict[output_node_name]
    
    return expected
    

In [18]:
model_unmodified = ModelWrapper("/tmp/quartznet_modified.onnx")

expected_unmodified = execute_unit_input_test(model_unmodified)
print("{}".format(expected_unmodified))

[[28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28
  28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28
  28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28
  28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28
  28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28 28
  28 28 28 28 28 28 28 28]]


In [19]:
model_modified = ModelWrapper("/tmp/quartznet_modified_4d.onnx")

expected_4d_model = execute_unit_input_test(model_modified)
print("{}".format(expected_4d_model))

[[[28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]
  [28]]]


In [1]:
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.core.modelwrapper import ModelWrapper

model_lowered = ModelWrapper("/tmp/quartznet_modified_4d.onnx")
#import pdb; pdb.set_trace()

model_lowered = model_lowered.transform(LowerConvsToMatMul(), make_deepcopy=True, cleanup=False, fix_float64=True)


Got past infer shapes


In [2]:
from finn.transformation.general import RemoveUnusedTensors

model_lowered = model_lowered.transform(RemoveUnusedTensors())

In [ ]:
expected_4d_lowered = execute_unit_input_test(model_lowered)
print("{}".format(expected_4d_lowered))

In [6]:
model_lowered.save("/workspace/finn/quartznet_lowered.onnx")

ValueError: Message ONNX_REL_1_7.ModelProto exceeds maximum protobuf size of 2GB: 3381322180

In [ ]:
model_lowered = ModelWrapper("/tmp/quartznet_lowered.onnx")

expected_lowered_model = execute_unit_input_test(model_lowered)
print("{}".format(expected_lowered_model))